### Домашняя работа "Коллаборативная фильтрация"

###### -Пакет SURPRISE
###### -Используйте данные MovieLens 1M
###### -Можно использовать любые модели из пакета получите RMSE на тестовом сете 0.87 и ниже


In [ ]:
!pip install surprise

     |████████████████████████████████| 11.8MB 273kB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp36-cp36m-linux_x86_64.whl size=1618283 sha256=3e83e5838716bf2e107521463a27b7ca7f94b524980a859955568bbf3f9ca688
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [4]:
from surprise import Dataset, accuracy, Reader, SVD, KNNBaseline
from surprise.model_selection import train_test_split, KFold
from statistics import mean
import pandas as pd

In [5]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv(r'ratings.csv')

In [8]:
movies_with_ratings = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)

In [9]:
dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})

In [10]:
dataset.head ()

,uid,iid,rating
0,1.0,Toy Story (1995),4.0
1,5.0,Toy Story (1995),4.0
2,7.0,Toy Story (1995),4.5
3,15.0,Toy Story (1995),2.5
4,17.0,Toy Story (1995),4.5


In [11]:
dataset.rating.min(), dataset.rating.max()

(0.5, 5.0)

In [12]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(dataset, reader)

In [15]:
trainset, testset = train_test_split(data, test_size=.70)

In [16]:
kf = KFold(n_splits=5)

###### SVD 

In [17]:
algo = SVD()

for trainset, testset in kf.split(data):
    algo.fit(trainset)
    predictions = algo.test(testset)
    accuracy.rmse(predictions, verbose=True)

RMSE: 0.8779
RMSE: 0.8683
RMSE: 0.8658
RMSE: 0.8721
RMSE: 0.8713


###### KNN

In [18]:
algo = KNNBaseline(k=30, sim_options={'name': 'pearson_baseline', 'user_based': False})

accuracy_n = []
n = 0
for trainset, testset in kf.split(data):
    algo.fit(trainset)
    predictions = algo.test(testset, verbose=False)
    n += 1
    accuracy_n.append(accuracy.rmse(predictions, verbose=False))
    print('RMSE: ', accuracy.rmse(predictions, verbose=False))

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE:  0.8506686097600642
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE:  0.8542447765927902
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE:  0.8585674283722767
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE:  0.8485218823273744
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE:  0.8500075654330331


Можно наблюдать, что результат лучше и мы получили необходимые значения.